In [14]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_4.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix{filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


#filename_sub = "4_sub_mat_3.csv"
#compute_submatrix(data, group_3, filename_sub, max_tau)

filename_cross = "4_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross)


submatrix4_cross_matrix_2_3.csv has been saved


,38099.0,38101.0,38103.0,38105.0,39001.0,39003.0,39005.0,39007.0,39009.0,39011.0,...,56027.0,56029.0,56031.0,56033.0,56035.0,56037.0,56039.0,56041.0,56043.0,56045.0
21083.0,-0.221814,0,0.188831,0,0.10608,0.204166,-0.19044,0.398868,0.227396,0.156857,...,-0.096043,-0.200458,0,0,0.114406,-0.341057,-0.306119,-0.228415,0.321267,0.081359
21085.0,0.249342,0.463902,0.22647,0.426863,0.310083,-0.403399,0.231527,0.071809,0.247164,0.437146,...,0.306736,0.243909,0.470737,0.340456,0.407364,0,0.154216,0.117639,-0.09315,0.466721
21087.0,-0.092573,0.282431,-0.204529,0.393729,0.247248,-0.441303,0.15357,-0.106009,0.125466,0.332298,...,-0.233319,0.160074,0.24267,0.292735,0.24213,0,0,0.159237,-0.599125,0.144367
21089.0,0.199137,0.525035,0.323016,0.554599,0.402279,-0.291487,0.315307,0,0.352316,0.416868,...,-0.239427,0.340369,0.554927,0.46,0.327003,0,0.170601,0.135756,-0.15641,-0.326348
21091.0,0.378989,-0.313548,0.275906,-0.371867,0.309253,0.17396,0.323286,0.182112,0.3583,0.318768,...,-0.322363,0.275319,0.41677,0.269745,0.358759,0.202788,0.337347,0.184214,0.175245,0.324504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38089.0,0.326725,0.544824,0.327502,0.610408,0.371108,0.39655,0.424367,-0.053008,0.417415,0.437038,...,-0.192246,0.353158,0.494849,0.494835,0.423116,0,0.158055,0.117802,-0.189823,0.388389
38091.0,-0.155621,0.119169,0.157885,0.13152,0.331989,-0.194445,-0.123554,0.156825,0.133426,-0.126853,...,0.131743,0.064325,0.082531,0.125557,-0.15364,-0.227071,-0.189031,0.132145,0.260117,-0.122055
38093.0,-0.123235,0.449785,-0.166091,0.544537,0.286757,-0.405929,0.210259,0,0.254206,-0.352987,...,-0.29118,0.198279,0.306636,-0.389756,0.370707,0,0.075653,0.13329,-0.096453,-0.255311
38095.0,0.422396,-0.519651,-0.472599,-0.545779,-0.324844,0.472769,-0.456094,-0.071587,-0.514648,0.44164,...,-0.349723,0.557488,0.617442,-0.483125,-0.466194,-0.291507,-0.287474,0.066046,0.195501,0.409093
